In [1]:
import uproot
import numpy as np
import os
import matplotlib.pyplot as plt

"""liczba = os.popen('cd new_traning/; find -type f | wc -l').read()
liczba2 = int(liczba)"""

"liczba = os.popen('cd new_traning/; find -type f | wc -l').read()\nliczba2 = int(liczba)"

In [227]:
import glob
"""file_pattern = "new_traning/*.root"
files = glob.glob(file_pattern)"""

file = uproot.open("/home/szymon/ML_ex/autoencoding_pp/new_traning/signal.root")
file2 = uproot.open("/home/szymon/ML_ex/autoencoding_pp/new_bkg/VV_org.root")
#file3 = uproot.open("/home/szymon/ML_ex/autoencoding_pp/new_traning/VV_bkg1.root")
#file4 = uproot.open("/home/szymon/ML_ex/autoencoding_pp/new_traning/VV_bkg2.root")

branch1 = file['nominal'].arrays(library="np")
branch2 = file2['nominal'].arrays(library="np")
#branch3 = file3['nominal'].arrays(library="np")
#branch4 = file4['nominal'].arrays(library="np")

KeyboardInterrupt: 

In [188]:

#branch_list = [uproot.open(files[i])['nominal'].arrays(library="np") for i in range (liczba2) ]

branch_list = [branch1, branch2, branch3, branch4]

In [189]:
combined_data = {}

In [190]:
dobre_zmienne = [
    "lep_Eta_0",
    "lep_Eta_1",
    "tau_eta_0",
    "tau_width_0",
    "tau_BDTJetScore_0",
    "tau_BDTJetScoreSigTrans_0",
    "tau_RNNJetScoreSigTrans_0",
    "DRll01",
    "lead_jetEta",
    "p_DR_l1_t1",
    "p_DR_l2_t1",
    "p_Dphi_l1_t1",
    "p_Dphi_l2_t1",
    "p_Deta_j1_j2",
    "p_DR_j1_j2",
    "p_Dphi_l1_j1",
    "p_Dphi_l2_j1",
    "p_Deta_l1_j1",
    "p_Deta_l1_j2",
    "p_Deta_l2_j1",
    "p_Deta_l2_j2",
    "p_DR_l1_j1",
    "p_DR_l1_j2",
    "p_DR_l2_j1",
    "p_drCloserJetToTau",
    "p_maxeta2leptons",
    "p_detaCloserJetToLeadLep",
    "p_drCloserJetToLeadLep",
    "p_detaFarestJetToTau",
    "p_drFarestJetToTau",
    "p_drFarestJetToLeadLep",
    "p_detaFarestJetToLeadLep",
    "p_angle_LeadLepTau",
    "p_angle_Lep1_Lep2",
    "p_angle_Lep2ToTau",
    "p_angle_LeadJetToTau",
    "p_angle_LeadJetToLeadLep",
    "p_angle_LeadJetToLep2",
    "p_angle_closetJetToL1",
    "p_angle_farestJetToTau",
    "p_angle_closetJetToL2",
    "p_angle_farestJetToL2",
    "p_drCloserJetToLep2",
    "p_drFarestJetToLep2",
    "p_LBoost2L_DiffTauLep1eta",
    "p_LBoost2L_AngleL2Tau",
    "p_LBoost2L_AngleL2J2",
    "p_LBoost2L_AngleTauJ1",
    "p_LBoost2L_drFarestJetToTau",
    "p_LBoost2L_angle_farestJetToL2",
    "p_LBoost2L_drCloserJetToLep2",
    "p_LBoost2L_drFarestJetToLep2",
    "p_LBoostL1Tau_lep2Theta",
    "p_LBoostL1Tau_tauTheta",
    "p_LBoostL1Tau_DiffTauLep2phi",
    "p_LBoostL1Tau_AngleL2Tau",
    "p_LBoostL1Tau_AngleL2J1",
    "p_LBoostL1Tau_AngleTauJ1",
    "p_LBoostL1Tau_angle_farestJetToTau",
    "p_LBoostL1Tau_angle_farestJetToL1",
    "p_LBoostL1Tau_angle_closetJetToL2",
    "p_LBoostL1Tau_drFarestJetToLep2",
    "p_LBoostL2Tau_lep1Theta",
    "p_LBoostL2Tau_tauTheta",
    "p_LBoostL2Tau_DiffTauLep1phi",
    "p_LBoostL2Tau_AngleL1Tau",
    "p_LBoostL2Tau_AngleL1J1",
    "p_LBoostL2Tau_AngleTauJ1",
    "p_LBoostL2Tau_DRL1J1",
    "p_LBoostL2Tau_drFarestJetToLeadLep",
    "p_LBoostL2Tau_angle_closetJetToTau",
    "p_LBoostL2Tau_angle_closetJetToL1",
    "p_LBoostL2Tau_angle_farestJetToTau",
    "p_LBoostL2Tau_angle_farestJetToL1",
    "p_LBoostL2Tau_angle_closetJetToL2",
    "p_LBoostL2Tau_angle_farestJetToL2"
]


In [191]:
dobre_zmienne2 = [
    "lep_sigd0PV_0",
    "lep_sigd0PV_1",
    "tau_BDTEleScoreSigTrans_0",
    "p_DR_l2_j2",
    "p_drCloserLepToTau",
    "p_detaCloserJetToTau",
    "p_detaCloserJetToLep2",
    "p_detaFarestJetToLep2",
    "p_LBoost2L_lep2Theta",
    "p_LBoost2L_DiffTauLep2phi",
    "p_LBoost2L_AngleL1Tau",
    "p_LBoost2L_AngleL2J1",
    "p_LBoost2L_DRL2Tau",
    "p_LBoost2L_DRL1J2",
    "p_LBoost2L_DRL2J1",
    "p_LBoost2L_DRL2J2",
    "p_LBoost2L_DRTauJ1",
    "p_LBoost2L_DRTauJ2",
    "p_LBoost2L_drCloserJetToLeadLep",
    "p_LBoost2L_drFarestJetToLeadLep",
    "p_LBoost2L_angle_closetJetToL1",
    "p_LBoost2L_angle_farestJetToTau",
    "p_LBoost2L_angle_closetJetToL2",
    "p_LBoostL1Tau_lep1Theta",
    "p_LBoostL1Tau_AngleL1L2",
    "p_LBoostL1Tau_AngleL1J1",
    "p_LBoostL1Tau_DRL2J1",
    "p_LBoostL1Tau_DRL2J2",
    "p_LBoostL1Tau_DRTauJ1",
    "p_LBoostL1Tau_DRTauJ2",
    "p_LBoostL1Tau_drCloserJetToTau",
    "p_LBoostL1Tau_drCloserJetToLeadLep",
    "p_LBoostL1Tau_drFarestJetToTau",
    "p_LBoostL1Tau_drFarestJetToLeadLep",
    "p_LBoostL1Tau_angle_closetJetToTau",
    "p_LBoostL1Tau_angle_closetJetToL1",
    "p_LBoostL1Tau_drCloserJetToLep2",
    "p_LBoostL2Tau_lep2Theta",
    "p_LBoostL2Tau_DiffLep1Lep2phi",
    "p_LBoostL2Tau_AngleL1L2",
    "p_LBoostL2Tau_AngleL2J1",
    "p_LBoostL2Tau_DRL1J2",
    "p_LBoostL2Tau_DRL2J2",
    "p_LBoostL2Tau_DRTauJ1",
    "p_LBoostL2Tau_DRTauJ2",
    "p_LBoostL2Tau_drCloserJetToTau",
    "p_LBoostL2Tau_drCloserJetToLeadLep",
    "p_LBoostL2Tau_drFarestJetToTau",
    "p_LBoostL2Tau_drCloserJetToLep2",
    "p_LBoostL2Tau_drFarestJetToLep2"
]

In [192]:
def delete_branches(branches):
    del branches['jet_isbtagged_DL1r_77']
    del branches['mc_kFactor']
    del branches['RunYear']
    del branches['mc_rawXSection']
    del branches['lep_ID_0']
    del branches['p_Event_No']
    del branches['jet_pt']
    del branches['jet_eta']
    del branches['jet_phi']
    del branches['jet_e']
    del branches['lep_vz_0']
    del branches['lep_vz_1']
    del branches['taus_DL1r_0']
    del branches['p_sumElepAll_EJet']
    del branches['p_sumPtleptauAll_Ptjet']
    del branches['best_Z_Mll']
    del branches['best_Z_other_MtLepMet']
    del branches['best_Z_other_Mll']
    del branches['minOSSFMll']
    del branches['minOSMll']
    del branches['totalEventsWeighted']
    del branches['weight_lumi_xsec']
    del branches['tau_charge_0']
    del branches['p_LBoostL1Tau_angle_farestJetToL2']
    del branches['eventNumber']
    del branches['p_is2lepTight']
    del branches['p_isTauSelected']
    del branches['tau_passEleBDT_0']
    del branches['tau_passMuonOLR_0']
    del branches['lep_ambiguityType_1']
    del branches['lep_ambiguityType_0']
    del branches['lep_plvWP_Tight_1']
    del branches['lep_plvWP_Tight_0']
    del branches['lep_isMedium_1']
    del branches['lep_isMedium_0']
    del branches['lep_isTightLH_1']
    del branches['lep_isTightLH_0']
    del branches['lep_d0_0'] # niekoniecznie usuniecie
    del branches['lep_BestMatchTrackEta_0']
    del branches['lep_BestMatchTrackPhi_0']
    del branches['lep_BestMatchTrackZ0_0']
    del branches['lep_BestMatchTrackQ_0']
    del branches['lep_chargeIDBDTLoose_0']
    del branches['lep_ID_1']
    del branches['lep_d0_1']
    del branches['lep_BestMatchTrackEta_1']
    del branches['lep_BestMatchTrackPhi_1']
    del branches['lep_BestMatchTrackZ0_1']
    del branches['lep_BestMatchTrackQ_1']
    del branches['lep_chargeIDBDTLoose_1']
    del branches['tau_numTrack_0']
    del branches['tau_passJetIDRNNMed_0']
    del branches['nJets_OR_DL1r_77']
    del branches['sublead_jetEta']
    del branches['sublead_jetPt']
    del branches['p_angle_LeadJetSubLeadJet']
    del branches['p_LBoost2L_tauPt']
    del branches['p_LBoost2L_DiffLep1Lep2phi']
    del branches['p_LBoost2L_AngleL1L2']
    del branches['p_LBoostL1Tau_DiffTauLep1phi']
    del branches['p_LBoostL1Tau_AngleL1Tau']
    del branches['p_LBoostL2Tau_DiffTauLep2phi']
    del branches['p_LBoostL2Tau_AngleL2Tau']
    del branches['lep_Phi_0']
    del branches['lep_EtaBE2_0']
    del branches['lep_z0_0']
    del branches['lep_deltaz0_0']
    del branches['lep_BestMatchTrackD0_0']
    del branches['lep_E_1']
    del branches['lep_Phi_1']
    del branches['lep_EtaBE2_1']
    del branches['lep_deltaz0_1']
    del branches['lep_BestMatchTrackD0_1']
    del branches['tau_phi_0']
    del branches['lead_jetPhi']
    del branches['sublead_jetPhi']
    del branches['p_Dphi_j1_j2']
    del branches['p_Dphi_l1_j2']
    del branches['p_Dphi_l2_j2']
    del branches['p_invMass_l2_j2']
    del branches['p_angle_closetJetToTau']
    del branches['p_angle_farestJetToL1']
    del branches['p_LBoostL1Tau_AngleL2J2']
    del branches['p_LBoostL2Tau_AngleL2J2']
    del branches['p_LBoostL2Tau_AngleTauJ2']
    for i in dobre_zmienne:
        del branches[i]
    '''for j in dobre_zmienne2:
        del branches[j]'''

In [193]:
# p_LBoost2L_tauPt 
# p_LBoost2L_DiffLep1Lep2phi
# p_LBoost2L_AngleL1L2
# p_LBoostL1Tau_DiffTauLep1phi
# p_LBoostL2Tau_DiffTauLep2phi
# 

In [194]:
for i in branch_list:
    delete_branches(i)


In [195]:
keys = list(branch_list[0])

In [196]:
def join_branchKey(arr):
    combo = arr
    if len(arr) > 1:
        combo = np.concatenate([arr[0], arr[1]])
        for i in range(len(arr)):
            combo = np.concatenate([combo, arr[i]])
    return combo

In [197]:
def join_branch(branch):
    try:
        for key in keys:
            if all(isinstance(item, np.ndarray) for item in branch[key]):
                branch[key] = join_branchKey(branch[key])
        return branch
    except IndexError:
        return branch

In [198]:
for i in range(len(branch_list)):
    branch_list[i] = join_branch(branch_list[i])

In [199]:
combined_data

{}

In [200]:
for i in keys:
    combo = np.concatenate([branch_list[0][i], np.array([])])
    for k in branch_list[1:]:
        combo = np.concatenate([combo, k[i]])
    combined_data[i] = combo
    try:
        combined_data[i] = combined_data[i].astype(np.float32)
    except ValueError:
        print(i)

In [201]:
combined_data2 = {}

In [202]:
for i in keys:
    combo = np.concatenate([branch_list[0][i],branch_list[1][i]], axis=None)
    combined_data2[i] = combo
print(len(combined_data2["lep_Pt_0"]))

26773


In [203]:
def data_before():
    ex = []
    std = []
    min = []
    max = []
    std5 = []
    
    for key in keys:
        ex.append(np.mean(combined_data[key]))
        std.append(np.std(combined_data[key]))
        min.append(np.min(combined_data[key]))
        max.append(np.max(combined_data[key]))
        std5.append(5*np.std(combined_data[key]))
        
    return ex, std, min, max, std5

In [204]:
for i in range(len(keys)):
    if np.min(combined_data[keys[i]]) == -999.0:
        del combined_data[keys[i]]

In [205]:
for key in keys:
    if "_inv" in key:
        for i in range(len(combined_data[key])):
            if combined_data[key][i] < 0:
                combined_data[key][i] = 0

In [206]:
keys = list(combined_data)

In [207]:
ex, std, min, max, std5 = data_before()

In [208]:
'''for i in range(len(keys)):
    print(f'{keys[i]}: Mean: {ex[i]}, Std: {std[i]}, Min: {min[i]}, Max: {max[i]}, Std5: {std5[i]}')
    print("---------------------------------------------------------------------------------------------------")'''

'for i in range(len(keys)):\n    print(f\'{keys[i]}: Mean: {ex[i]}, Std: {std[i]}, Min: {min[i]}, Max: {max[i]}, Std5: {std5[i]}\')\n    print("---------------------------------------------------------------------------------------------------")'

In [209]:
def paint(m, lst, color, key):
    fig = plt.figure()
    biny = bins(m, lst)
    plt.hist(lst, bins=biny, color=color)
    plt.title(key)
    return fig

In [210]:
def bins(n, list):
    bins = []

    list = sorted(list)

    last = list[-1]
    k = np.min(list)
    while k < last:
        bins.append(k)
        k += (last - list[0])/n


    return bins

In [211]:
def cuting(list):
    mean = np.mean(list)
    std = np.std(list)
    
    filtred = []
    
    max_list = []
    min_list = []
    
    max_plus = mean + 5*std
    max_minus = mean - 5*std
    for i in range(len(list)):
        if(list[i] <= max_plus and list[i] >= max_minus):
            filtred.append(list[i])
        elif (list[i] >= max_plus):
          max_list.append(list[i])
        elif (list[i] <= max_minus):
            min_list.append(list[i])
    if (len(max_list) > 0):
        cut_max = np.min(max_list)
    else:
        cut_max = np.max(filtred)
    if(len(min_list) > 0):
        cut_min = np.max(min_list)
    else:
        cut_min = np.min(filtred)
    
    for i in range(len(max_list)):
        filtred.append(max_plus)
    
    for i in range(len(min_list)):
        filtred.append(max_minus)
    
    return [np.asarray(filtred), cut_max, cut_min, max_list, min_list]

In [212]:
'''for key in keys:
    cut = cuting(combined_data[key])
    print(f'{key} -- filtered: {len(cut[0])} right cut num: {len(cut[3])}, left cut num: {len(cut[4])}')'''

"for key in keys:\n    cut = cuting(combined_data[key])\n    print(f'{key} -- filtered: {len(cut[0])} right cut num: {len(cut[3])}, left cut num: {len(cut[4])}')"

In [213]:
def minus_value_filter(keys, branch):
    try:
        for key in keys:
            min = np.min(branch[key])
            max = np.max(branch[key])
            distance = abs(max - min)
            if min < 0:
                for i in range(len(branch[key])):
                    branch[key][i] = branch[key][i] + distance
        return branch
    except ValueError:
        print(key)

In [214]:
combined_data = minus_value_filter(keys, combined_data)


In [215]:
def norm(keys, branch):
    
    maxes = {}
    mins = {}
    
    for key in keys:
        max = np.max(branch[key])
        min = np.min(branch[key])
        distance = max - min
        
        for i in range(len(branch[key])):
            if distance != 0:
                branch[key][i] = (branch[key][i] - min) / distance
        if(max != 0):
            maxes[key] = (max - min) / distance   
            mins[key] = 0
        
        
    return [branch, maxes, mins]

In [216]:
combined_data = norm(keys, combined_data)[0]

In [217]:
globals_max = {}
globals_min = {}

for key in keys:
    
    cut = cuting(combined_data[key])
    combined_data[key] = cut[0]
    globals_max[key] = cut[1]
    globals_min[key] = cut[2]


In [218]:
'''ex, std, min, max, std5 = data_before()

for i in range(len(keys)):
    print(f'{keys[i]}: Mean: {ex[i]}, Std: {std[i]}, Min: {min[i]}, Max: {max[i]}, Std5: {std5[i]}')
    print("---------------------------------------------------------------------------------------------------")'''

'ex, std, min, max, std5 = data_before()\n\nfor i in range(len(keys)):\n    print(f\'{keys[i]}: Mean: {ex[i]}, Std: {std[i]}, Min: {min[i]}, Max: {max[i]}, Std5: {std5[i]}\')\n    print("---------------------------------------------------------------------------------------------------")'

In [219]:
def paint2(key, m, lst, lst2, color, color2, alpha):
    fig = plt.figure()
    biny = bins(m, lst[key])
    plt.hist(lst[key], bins=biny, color=color)
    plt.hist(lst2[key], bins=biny, color=color2, alpha=alpha)
    plt.title(key)
    return fig

In [220]:
list_keys = []
filtered_cor = []

sum = 0
for key in keys:
    a = combined_data[key]
    list_keys.append(key)
    for k in keys:
        try:
            if k != key and k not in list_keys:
                b = combined_data[k]
                cor_coff = np.corrcoef(a.reshape(1, -1), b.reshape(1, -1))
                if abs(cor_coff[0, 1]) > 0.8:
                    print(f'{key} with {k}: {cor_coff[0, 1]}')
                    sum += 1
                    if key not in filtered_cor:
                        filtered_cor.append(key)
        except AttributeError:
            print(f'{key}: {a}')
            print(f'{k}: {b}')
        
                
print(sum)
print(filtered_cor)
print(len(filtered_cor))

p_Deta_l1_l2 with p_LBoost2L_lep1Theta: -0.9342819192873532
p_Deta_l1_l2 with p_LBoost2L_lep2Theta: 0.9342482095232069
p_Deta_l1_t1 with p_LBoost2L_tauTheta: 0.8012322808918452
p_Deta_l1_t1 with p_LBoostL1Tau_lep1Theta: -0.913516241570053
p_Deta_l2_t1 with p_LBoostL2Tau_lep2Theta: -0.9097280896840807
p_detaCloserLepToTau with p_LBoost2L_tauTheta: 0.8041307061558616
p_LBoost2L_lep1Pt with p_LBoost2L_lep2Pt: 0.9999999993271049
p_LBoost2L_lep1Theta with p_LBoost2L_lep2Theta: -0.9994589153954411
p_LBoost2L_tauTheta with p_LBoostL1Tau_lep1Theta: -0.805184347419741
p_LBoost2L_DiffTauLep1phi with p_LBoostL1Tau_DiffLep1Lep2phi: 0.8648844404013759
p_LBoost2L_AngleL1J1 with p_LBoost2L_AngleL2J1: -0.9991063736339486
p_LBoost2L_AngleL1J2 with p_LBoostL1Tau_AngleL1J2: 0.914423265655642
p_LBoost2L_AngleL1J2 with p_LBoostL2Tau_AngleL1J2: 0.8187007874055784
p_LBoost2L_AngleTauJ2 with p_LBoostL1Tau_AngleTauJ2: 0.8609003160243873
p_LBoost2L_drCloserJetToTau with p_LBoost2L_angle_closetJetToTau: 0.844333

In [221]:
fil_keys = []

for key in keys:
    if key not in filtered_cor:
        fil_keys.append(key)
        
print(len(fil_keys))

114


In [222]:
keys = fil_keys

In [223]:
signal = branch_list[0]
background = {}
for key in keys:
    background[key] = combined_data[key][len(signal[key]):]
    signal[key] = combined_data[key][:len(signal[key])]


In [224]:
signal_filtered = {}
background_filtered = {}
combined_data_filtered = {}

In [225]:
for key in keys:
    signal_filtered[key] = signal[key]
    background_filtered[key] = background[key]  
    combined_data_filtered[key] = combined_data[key]

In [226]:
np.save("new_signal_data.npy", signal_filtered)
np.save("new_background_data.npy", background_filtered)
np.save("new_combined_data.npy", combined_data_filtered)

In [ ]:
for key in keys:
    paint2(key, 100, background_filtered, signal_filtered, "blue", "red", 0.6)